In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

silicon = Material(Materials.get_by_name_first_match('Silicon'))



In [ ]:
from utils.visualize import visualize_materials
visualize_materials(silicon)

## Create a silicon nanowire

In [ ]:
from mat3ra.made.tools.modify import filter_by_cylinder, rotate, filter_by_box, add_vacuum_sides
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration


slab_config = SlabConfiguration(bulk=silicon, miller_indices=(1,1,0), thickness=2, vacuum=0, use_orthogonal_z=True, xy_supercell_matrix=[[3,0],[0,2]]) 
silicon_slab = create_slab(slab_config)

visualize_materials(silicon_slab)
visualize_materials(silicon_slab, rotation="-90x")

rotated_slab = rotate(silicon_slab, "z", -45)
silicon_nanowire = filter_by_box(rotated_slab, [0.2,0.28,0], [0.78,0.87,1])
visualize_materials(silicon_nanowire)
visualize_materials(silicon_nanowire, rotation="-90x")
rotated_nanowire = rotate(silicon_nanowire, "z", -45)
# rotated_nanowire = rotate(rotated_nanowire, "x", 90, rotate_cell=True)
rotated_nanowire = add_vacuum_sides(rotated_nanowire, 10.0, on_x=True, on_y=True)
visualize_materials(rotated_nanowire)
visualize_materials(rotated_nanowire, rotation="-90x")

## Add passivation

In [ ]:
from mat3ra.made.tools.build.passivation import PassivationConfiguration, CoordinationBasedPassivationBuilder, create_passivation, get_unique_coordination_numbers
from mat3ra.made.tools.build.passivation.builders import CoordinationBasedPassivationBuilderParameters

passivation_config = PassivationConfiguration(
    slab=rotated_nanowire,
    passivant="H",
    bond_length=1.1,
)

params = CoordinationBasedPassivationBuilderParameters(shadowing_radius=3.9, coordination_threshold=10)
passivation_builder = CoordinationBasedPassivationBuilder(params)
print(get_unique_coordination_numbers(passivation_config, params))

passivated_nanowire = create_passivation(passivation_config, passivation_builder)

visualize_materials(passivated_nanowire)
visualize_materials(passivated_nanowire, rotation="-90x")
visualize_materials(passivated_nanowire, rotation="-90x,90y")

In [ ]:
from utils.jupyterlite import set_materials
passivated_nanowire.name = "H-Passivated Silicon Nanowire"
set_materials([passivated_nanowire])